## Coincidencias Organizaciones

In [1]:
!pip install unidecode

     |████████████████████████████████| 241 kB 5.2 MB/s 


In [ ]:
import os
import sys

import difflib
import pandas as pd
import importlib as imp
import unidecode

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')


module_path = '/content/drive/MyDrive/Colab Notebooks/JusticIA_desaparecidos/src'


if module_path not in sys.path:
    sys.path.append(module_path)

import config as cfg
import similar_words as sim

def reload():

  for lib in [cfg, sim]:
    imp.reload(lib)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Cargando transcripciones generadas automaticamente

In [ ]:
df_transcripts = sim.read_transcripts()

In [ ]:
# Seleccionando un ejemplo
text = df_transcripts.iloc[2, 3]
text

'DO mati] Exp-100-10-1-67. |\n— AVIDA, Faustóé 2 ko? 1-23,\n— 21.Abril.,-67.-Mie bro del CONSEJO DEI AUTODIFENS\n1 DEL PUEBLO 9E UCUALA, Gro, El día 19 actual en\n\n- Chilpancingo.Gro, los estudiantes de la liormel —\n; Mural de Ayotzinapa Cro, efectuaron Mitin en el\n: muÑal éste elemento en uso de la palabra solicítc\n— la Libortad de los Estudiambes Presos en lich,,\n| Pue. y Ceracruz, “a democratizacion de la Ehse\n fanmza y la implantación efectiva de Servicios —\n; Asistenciales, demandas que el alumnado de las\n— Normales de todo el Pais está haciendo patentes\n* con motivo de la Jornada Auspiciada por la Ceii.\n\n- EoDoo'

In [ ]:
# Leyendo organizaciones
df_orgs = sim.read_organizations()
df_orgs.head()

b'Skipping line 90: expected 1 fields, saw 3\nSkipping line 108: expected 1 fields, saw 10\nSkipping line 110: expected 1 fields, saw 3\nSkipping line 113: expected 1 fields, saw 4\nSkipping line 127: expected 1 fields, saw 3\nSkipping line 195: expected 1 fields, saw 5\nSkipping line 196: expected 1 fields, saw 2\nSkipping line 199: expected 1 fields, saw 9\nSkipping line 206: expected 1 fields, saw 8\nSkipping line 251: expected 1 fields, saw 2\nSkipping line 261: expected 1 fields, saw 2\nSkipping line 381: expected 1 fields, saw 3\nSkipping line 384: expected 1 fields, saw 2\nSkipping line 401: expected 1 fields, saw 2\nSkipping line 403: expected 1 fields, saw 2\nSkipping line 408: expected 1 fields, saw 2\nSkipping line 426: expected 1 fields, saw 4\nSkipping line 439: expected 1 fields, saw 2\nSkipping line 440: expected 1 fields, saw 2\nSkipping line 445: expected 1 fields, saw 3\nSkipping line 446: expected 1 fields, saw 2\nSkipping line 449: expected 1 fields, saw 2\nSkipping

,ORGANIZATIONS
0,partido comunista mexicano
1,liga comunista 23 de septiembre
2,asociacion civil emiliano zapata
3,fuerzas armadas revolucionarias
4,fuerzas armadas de liberacion


### Procesamiento de las organizaciones

In [ ]:
orgs_list = df_orgs['ORGANIZATIONS '].to_list()
len(orgs_list)

4648

In [ ]:
orgs_dict = {}

for org in orgs_list:
  orgs_dict[org] = sim.filter_sentence(org, camel_case=True)

In [ ]:
sim.to_pickle(orgs_dict, '/content/drive/MyDrive/Colab Notebooks/JusticIA_desaparecidos/data/orgs_processed_dict.p')

In [ ]:
orgs_dict = sim.read_pickle('/content/drive/MyDrive/Colab Notebooks/JusticIA_desaparecidos/data/orgs_processed_dict.p')

In [ ]:
orgs_dict['comite ejecutivo central proporcional']

['Comite', 'Ejecutivo', 'Central', 'Proporcional']

### Usando la clase TargetWords

In [ ]:
tw = sim.TargetWords(text, 'prosecuted')

In [ ]:
text

'DO mati] Exp-100-10-1-67. |\n— AVIDA, Faustóé 2 ko? 1-23,\n— 21.Abril.,-67.-Mie bro del CONSEJO DEI AUTODIFENS\n1 DEL PUEBLO 9E UCUALA, Gro, El día 19 actual en\n\n- Chilpancingo.Gro, los estudiantes de la liormel —\n; Mural de Ayotzinapa Cro, efectuaron Mitin en el\n: muÑal éste elemento en uso de la palabra solicítc\n— la Libortad de los Estudiambes Presos en lich,,\n| Pue. y Ceracruz, “a democratizacion de la Ehse\n fanmza y la implantación efectiva de Servicios —\n; Asistenciales, demandas que el alumnado de las\n— Normales de todo el Pais está haciendo patentes\n* con motivo de la Jornada Auspiciada por la Ceii.\n\n- EoDoo'

In [ ]:
' '.join(tw.sentence)

'mati Exp-10-10-1-67 Avida Faustoe 1-23 21.Abril. -67.-Mie bro Consejo Dei Autodifens Del Pueblo Ucuala Gro dia actual Chilpancingo.Gro estudiantes liormel Mural Ayotzinapa Cro efectuaron Mitin muNal elemento uso palabra solicitc Libortad Estudiambes Presos lich Pue Ceracruz democratizacion Ehse fanmza implantacion efectiva Servicios Asistenciales demandas alumnado Normales Pais haciendo patentes motivo Jornada Auspiciada Cei EoDo'

In [ ]:
tw.results

[(1.0, 'juan', ['Juan']),
 (1.0, '(a) juan', ['Juan']),
 (0.92, 'juan pestana', ['Juan', 'Pestana'])]

In [ ]:
# Búsqueda de organizaciones
tw = sim.TargetWords(text, 'organization')
tw.results

[(0.95,
  'consejo de autodefensa del pueblo',
  ['Consejo', 'Autodefensa', 'Pueblo']),
 (0.86, 'los procesos', ['Procesos'])]

In [ ]:
# Búsqueda de lugares (Estados)
tw = sim.TargetWords(text, entity='state')
tw.results

OrderedDict([('Puebla', ('Pueblo', 0.83)),
             ('Veracruz', ('Ceracruz', 0.88)),
             ('Gro', ('Gro', 1.0)),
             ('Pue', ('Pue', 1.0))])

### Prueba de automtización

In [ ]:
reload()

In [ ]:
df_transcripts = sim.read_transcripts()

In [ ]:
df_transcripts['TextoLimpio'] = ''
df_transcripts['Organizaciones'] = ''

results_dict = {}

for i in df_transcripts.index[20:50].to_list():
  text = df_transcripts.iloc[i, 3]
  tw = sim.TargetWords(text, 'organization')
  df_transcripts.at[i, 'TextoLimpio'] = ' '.join(tw.sentence)
  df_transcripts.at[i, 'Organizaciones'] = tw.results

In [ ]:
df_transcripts.loc[20:50, ['Texto', 'MetodoTexto', 'TextoLimpio', 'Organizaciones']]

,Texto,MetodoTexto,TextoLimpio,Organizaciones
20,"7XÁN hacia ía Población de Poza Honda para com-\norar café y esperando los guerrilleros Su regre-\nSO aproximadamente a las tres de la tarde donde-\n1e hace la varada y el propio declarante apunta\ndole con un fusil 30 M-l lo hace detenerse apun-\nt andole directamente. que asimisto Salomon RÍ05\n(a) Ramón portando UN fhsil 30-=M-2 se quedó ---\n amagando al secuestrado mientras Humberto RIVERA\nLEYVA Sa) +saias, con uns pistola 38 super auto\n“abrio la puerta del automóvil se dice camioneta-\n— para bajar al secuestrado CUAUHTEMOC GARCIA Ti\nRAN: que el tal la Victor con una carabina ---\n— 30- Ml se dedizó a bajar de la parte de atrás =\nde la camioneta a los dos cargadores que viaja-\n— te de |: camioneta y al mando con una carabina -",automatico,7Xan hacia Poblacion Poza Honda com- orar cafe esperando guerrileros regre- aproximadamente tres tarde donde- hace varada propio declarante apunta dole fusil M-l hace detenerse apun- andole directamente asimisto Salomon Ri05 Ramon portando fhsil 30-=M-2 quedo amagando secuestrado mientras Humberto Rivera Leyva +saias uns pistola super auto abrio puerta automovil dice camioneta- bajar secuestrado Cuauhtemoc Garcia Ran tal Victor carabina 30- dedizo bajar parte atras camioneta dos cargadores viaja- camioneta mando carabina,None
21,"den a la ciudad de léxico D.T.para que tanto en—\nlas oficinas de la Cómisión del Río Balsas: como- .\nen la Cámara de Senadores conozcan al Senador -- —\nRUBEN FIGUEROA, lo vigilen y estudien sus habitos\n“y costumbres para que de ser posible realicen --\nel .ecuestro en la ciudad de México yDoFo, pero =\n— consideraron posteriormente que sería dificil --\n— ésto en razón de que el Senador andaba muy custo=\n diado y en la ciudad de Néxico existía mucho pa- '\n trullaje y después de 8 días deciden regresar -- |\na la sierra de Guerrero para reunirse con la Bri-\nSada. Que del mes de enero a abril de 1974, el de\ndla voz junto con el grupo se dedicó solamente ---\n-a la labor de politización en diferentes pueblos\n— “el Mpio.de Atoyac de Alvarez Y reclytar además -\nelementos propicios para la gueórilla y fusión --",automatico,den ciudad lexico D.T.para oficinas Comision Rio Balsas como- Camara Senadores conozcan Senador Ruben Figueroa vigilen estudien habitos costumbres ser posible realicen .ecuestro ciudad Mexico yDoFo consideraron posteriormente seria dificil razon Senador andaba custo= diado ciudad Nexico existia pa- trulaje despues dias deciden regresar sierra Guerrero reunirse Bri- Sada Que mes enero abril 1974 dla voz junto grupo dedico solamente labor politizacion diferentes pueblos Mpio.de Atoyac Alvarez reclytar ademas elementos propicios gueorila fusion,"[(1.0, comision n, [Comision]), (1.0, comision del rio balsas, [Comision, Rio, Balsas]), (1.0, camara de senadores, [Camara, Senadores]), (0.86, el mexicano, [Mexicano])]"
22,"HERNANDEZ GOMEZ,Pedro — — xP. 100-10-16-4-55\nE To H-2% 1-10“\no | 18> Et a o\nala Brigada. Que aprincipios del mes de abril.\n- en un lugar cércano al Cacao se encontraban el -\n| de la voz. responsable deun grupo aproximado de\n— 20 guerrilleros son informados que esos días ha=-\n| bian hido a la población mencionada dos agentes-.\nde la policia judicial, dandoles los generales de\nestos dos Supuestos agentes y decidiendo el de ls\n— YOZ “omo responsable junto Con su grupo,ejecutar\n—a dichos agentes, apostandose a los lados del cas\n— 16 ombres con el de la vOZ , todos -—.",automatico,Hernandez Gomez Pedro 10-10-16-4-5 H-2 1-10 ala Brigada Que aprincipios mes abril lugar cercano Cacao encontraban voz responsable deun grupo aproximado guerrileros informados dias ha=- bian hido poblacion mencionada dos agentes- policia judicial dandoles generales dos Supuestos agentes decidiendo Yoz omo responsable junto Con grupo ejecutar dichos agentes apostandose lados cas ombres vOZ,"[(0.87, policia judicial del estado, [Policia, Judicial]), (0.87, policia judicial, [Policia, Judicial]

In [ ]:
# df_transcripts['TextoLimpio'] = ''
df_transcripts['Estados'] = ''

results_dict = {}

for i in df_transcripts.index.to_list():
  text = df_transcripts.iloc[i, 3]
  tw = sim.TargetWords(text, 'state')
  df_transcripts.at[i, 'TextoLimpio'] = ' '.join(tw.sentence)
  df_transcripts.at[i, 'Estados'] = tw.results

In [ ]:
df_transcripts[['TextoLimpio', 'Estados']].sample(4)

,TextoLimpio,Estados
566,Brigada Campesina Ajusticiamiento Exp-10-10-16-4- H-259 L=10 =75.| pos N4- nero=75 .-Con objeto aprehender personas Ninvolucrudas actividades tipo subversivo madrugada mentos Pirecion apo- vados miembros jto dela Pol.Jud.del do.de Aguascalientes realizadron operacion sen Eji Venaderos Mpyio.de Yesus Mariao IVicha operacion concluyo 5.0 hs.habiendo isido detenidos Rodolfo Ramirez Fonce Ramone fosorro Munoz orrilo Medo dos anteriores buscados autorit 'dades porsu particiacon grupo guerrilera *rof Lucio Caban Barrientos.,"{'Aguascalientes': ('Aguascalientes', 1.0)}"
899,Universidad Autonoma Expo 10-10-1-78 Guerer 0.- H-206 L-74 Escuela Enfermeria No1 informo ineorpo racion condicionada Instituto Lazaro Cerdenas TIndio informe Comision Finenciera semana proxima daran conocer resultados auditoria realizada gestion ex-Rector QoBoPo Apolinar Arquimedez Morales Caranza Que Dro Wences Reza tendra un2 reunion Secretaria Pro gremacion Presupuesto respecto subsidio Urc conocer cuantos milones pesos gastaron en= 2dministracion anterior maximo lantelo-5e 207- mayoria discutir reunion reglementecion exemenes Universidado,"{'Guerrero': ('Guerer', 0.86)}"
168,Tae _._ Lel Pueblo Day hal la+ pets Son Ado Est Nto ASianL Civil Doranl Atico Ligero activi dano Mlomalico tis Art Apa o.0 ent aciM use itiene conc ando eciben inst moviliza 161 Cora ecaovsa aniZza Co. Dar Dava Jal Nza noJ Dog ecion Vas Ques Tem Actnato ecion tro E-Vudo Sub Los falo ane nanco Otro Mar Ala Anc homo meo Weno Sue nombre Istdor ado Tea Nevos Esta idue ceno 1Acl EaDacis maneja gue lema Cal Actesco man maneja Cacique ive Paciscu cui Tai Eur Poido Meta Comano Tuxdans descuhbie Ncepiton Silts 1/0.2 nar Voscerio cuandi di-igioa tenecia Sojas loexe hui sie jue Dem Eno T-a Vazquez Sojas cero Mim Nales cunada Vaz,"{'Puebla': ('Pueblo', 0.83), 'Jal': ('Jal', 1.0), 'Son': ('Son', 1.0)}"
334,-Genaro Dominguez Maldonado Expe1-1609-73._ H-.50 1-39 18-Jdun-73.-El Actl.visitado Lousdas Len Doza Gaytan Homero Froylan Cabalero Ramos Expo1-169=73 H68 L=39 25-Junio=73.-El actual visitado penal pors Jorce Mendoza Gaytan Expol1=16973 H.184 1-39 JeJul=73- actual visitado penal Arturo Caranza Conzalez Jorge Arturo Mendoza Caytan Consuelo Gaytan Gomez Dario Comez Gonzalez Espe- Ranza Parker Vazquez,None


In [ ]:
i

6684

In [ ]:
i = None
for i in range(1000000):
  print(i)

Streaming output truncated to the last 5000 lines.
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874

KeyboardInterrupt: ignored